In [3]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ШК], [Код склада], [ID Сайта], [Товарное направление],
[Товарная группа], [Проба], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Скидка на изделии], [Запрет скидки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ДК', N'БК', N'ЦБ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock.drop(columns=['8', 'Код объекта', 'Код склада'], inplace=True)

In [4]:
stock = stock[stock['ID Сайта'] != '']

In [5]:
ost = stock.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()

In [6]:
ost = ost[ost['Остаток'] >= 10]

In [7]:
stock.drop(columns=['Остаток'], inplace=True)

In [8]:
stock = stock.merge(ost, how='left', on='ID Сайта')

In [9]:
del ost

In [10]:
stock['Итоговая цена'] = np.where(
    stock['Запрет скидки'] == 'Нет', stock['Цена Розн., за шт'] * (1 - stock['Скидка на изделии'] / 100) * 0.5, stock['Цена Розн., за шт']
)

In [11]:
stock.drop(columns=['Цена Розн., за шт', 'Скидка на изделии', 'Запрет скидки'], inplace=True)

In [12]:
price_skidki = stock.groupby(by = 'ID Сайта', as_index=False)['Итоговая цена'].agg({'min', 'max'})
price_skidki.rename(columns={'max': 'Цена со скидками макс', 'min': 'Цена со скидками мин'}, inplace=True)
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 
stock = stock.merge(price_skidki, how='left', on='ID Сайта')

In [13]:
stock['Чистый вес'] = stock['Чистый вес'].astype(float)
stock = stock[stock['Чистый вес'] > 0]
clear_weight = stock.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'max'})
clear_weight.rename(columns={'max': 'Чистый вес макс', 'min': 'Чистый вес мин'}, inplace=True)
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 
stock = stock.merge(clear_weight, how='left', on='ID Сайта')

In [14]:
stock['Размах по цене со скидками'] = np.where(
    stock['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        stock['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

stock['Размах по весу'] = np.where(
    stock['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        stock['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [15]:
stock.drop(columns=['ШК', 'Итоговая цена', 'Чистый вес'], inplace=True)

In [16]:
stock.drop_duplicates(subset=['ID Сайта'], inplace=True)

In [17]:
stock = stock.query('Остаток >= 10')

In [20]:
stock.drop(columns=['Чистый вес'], inplace=True)

In [26]:
stock.sort_values(by=['Чистый вес размах, %', 'Цена со скидками размах, %'], ascending=False).to_excel('выборка с сильным разбросом цены и веса.xlsx', index=False)

In [29]:
top10 = stock.sort_values(by=['Цена со скидками размах, %'], ascending=False)[:10]

In [35]:
ids = top10['ID Сайта'].tolist()

In [31]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ШК], [ID Сайта], [Цена Розн., за шт], [Скидка на изделии], [Запрет скидки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ДК', N'БК', N'ЦБ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [32]:
stock['Итоговая цена'] = np.where(
    stock['Запрет скидки'] == 'Нет', stock['Цена Розн., за шт'] * (1 - stock['Скидка на изделии'] / 100) * 0.5, stock['Цена Розн., за шт']
)

In [36]:
stock.rename(columns={'ID Сайта': 'article'}, inplace=True)

In [37]:
stock = stock.query('article in @ids')

In [39]:
stock.rename(columns={'article': 'ID Сайта'}, inplace=True)
stock.drop(columns=['Цена Розн., за шт'], inplace=True)

In [40]:
stock.to_excel('в разрезе ШК с наибольшими отклонениями по цене.xlsx', index=False)

In [8]:
df = pd.read_csv(r'C:\Users\Trenkin.Sergey\Desktop\dldb\_Remainings_8h__202407171156.csv')

In [5]:
a = df.groupby(by = 'Товарное направление', as_index=False)['ID Сайта'].value_counts()

In [7]:
del df

In [6]:
a.groupby(by='Товарное направление', as_index=False)['count'].sum()

,Товарное направление,count
0,БИЖУТЕРИЯ,1130
1,БК,124947
2,БУ,2738
3,ВЫРИЦА,8832
4,ДК,243777
5,ДФРС,47001
6,ИФ,227070
7,ПДК,43386
8,СИ,863120
9,ФУТЛЯРЫ,218281


In [9]:
a['count'].sum()

1956344